In [1]:
import cv2
import os
import numpy as np
import dlib
import pandas as pd
def load_dataset(dataset_path, target_size=(100, 100)):
    images = []
    labels = []

    for label in os.listdir(dataset_path):
        label_path = os.path.join(dataset_path, label)
        for image_file in os.listdir(label_path):
            image_path = os.path.join(label_path, image_file)
            image = cv2.imread(image_path)
            
            # Check if the image is None (failed to read)
            if image is not None:
                # Resize the image to a consistent size
                image = cv2.resize(image, target_size)
                images.append(image)
                labels.append(label)

    return np.array(images), np.array(labels)


dataset_path = 'drowsiness_dataset'
images, labels = load_dataset(dataset_path)



In [2]:
labels

array(['ClosedFace', 'ClosedFace', 'ClosedFace', ..., 'OpenFace',
       'OpenFace', 'OpenFace'], dtype='<U10')

In [3]:
labels.size

2423

In [4]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")


indices_no_faces = []

def detect_facial_landmarks(image, index):
    global a, indices_no_faces  # Declare 'a' and 'indices_no_faces' as global variables

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)
    
    if not faces:
        # store not detected faces
        indices_no_faces.append(index)

    landmarks = []
    for face in faces:
        shape = predictor(gray, face)
        landmarks.append(shape)

    return landmarks

# Assuming 'images' is a list of loaded images

facial_landmarks = [detect_facial_landmarks(image, index) for index, image in enumerate(images)]

# print("Total images with no faces:", a)
# print("Indices of images with no faces:", indices_no_faces)


In [5]:
len(facial_landmarks)


2423

In [6]:
# Remove empty lists from facial_landmarks

facial_landmarks = [landmarks for landmarks in facial_landmarks if landmarks]

# Now, facial_landmarks contains only non-empty lists of facial landmarks


In [7]:
facial_landmarks

[[<_dlib_pybind11.full_object_detection at 0x21535c46d30>],
 ...]

In [8]:
len(facial_landmarks)

2217

In [11]:
# import pickle

# # Save facial_landmarks to a file
# with open('facial_landmarks.pkl', 'wb') as file:
#     pickle.dump(facial_landmarks, file)

# # Load facial_landmarks from the file
# with open('facial_landmarks.pkl', 'rb') as file:
#     facial_landmarks = pickle.load(file)

In [12]:
labels.size

2423

In [20]:
#  delete indexes of not detected faces

labels = np.delete(labels, indices_no_faces)

In [21]:
from sklearn.preprocessing import LabelEncoder
# convert labels from string to 0 and 1's

label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

print(labels)

[0 0 0 ... 1 1 1]


In [22]:
labels.size

2217

In [23]:

def eye_aspect_ratio(eye):
    # Extract coordinates of vertical eye landmarks
    v1 = np.array([eye[1].x, eye[1].y])
    v5 = np.array([eye[5].x, eye[5].y])
    v2 = np.array([eye[2].x, eye[2].y])
    v4 = np.array([eye[4].x, eye[4].y])

    # Extract coordinates of horizontal eye landmarks
    v0 = np.array([eye[0].x, eye[0].y])
    v3 = np.array([eye[3].x, eye[3].y])

    # Calculate the Euclidean distances
    vertical_dist1 = np.linalg.norm(v1 - v5)
    vertical_dist2 = np.linalg.norm(v2 - v4)
    horizontal_dist = np.linalg.norm(v0 - v3)

    # Calculate the EAR
    ear = (vertical_dist1 + vertical_dist2) / (2.0 * horizontal_dist)
    
    return ear


# Assuming 'facial_landmarks' is a list of facial landmarks for each image

lear=[]
rear=[]
for landmarks in facial_landmarks:
    for shape in landmarks:
        
        left_eye = shape.parts()[36:42]  # 36-41 are the indices of the left eye landmarks
        right_eye = shape.parts()[42:48]  # 42-47 are the indices of the right eye landmarks

        left_ear = eye_aspect_ratio(left_eye)
        right_ear = eye_aspect_ratio(right_eye)


        lear.append(left_ear)
        rear.append(right_ear)


lear = np.array(lear)
rear = np.array(rear)

In [24]:
type(lear[0])

numpy.float64

In [25]:
rear

array([0.16666667, 0.0625    , 0.14544633, ..., 0.28797019, 0.31497976,
       0.26923077])

In [26]:
# Create a DataFrame

data = {'lear': lear, 'rear': rear, 'Close/Open':labels}
data_frame = pd.DataFrame(data)

# Save the DataFrame to a CSV file

data_frame.to_csv('drowsi.csv', index=False)

In [27]:
data_frame['lear'][0]

0.13736056394868904